Groupby the neighborhood blocks by old way--- ramps scores

In [6]:
import json
import geojson
from shapely.geometry import shape, Point, Polygon
import geopandas as gpd
import folium


ramps_gdf = gpd.read_file('../data/Ramps New.json')
sidewalks_gdf = gpd.read_file('../data/Sidewalks.json')
ramps_gdf = ramps_gdf.to_crs(sidewalks_gdf.crs)
ramps_gdf.geometry = ramps_gdf.geometry.buffer(5)

joined_gdf = gpd.sjoin(sidewalks_gdf, ramps_gdf, op='intersects')
grouped_gdf = joined_gdf.groupby('SWK_ID').agg({'FID_left': list, 'score_1': 'mean'}).reset_index()
result_gdf = sidewalks_gdf.merge(grouped_gdf, on='SWK_ID', how='left')


new_gdf = result_gdf
# Create a map centered at a specific location
m = folium.Map(location=[42.361145, -71.057083], zoom_start=30)

# Add sidewalks layer to the map with color ramp based on average score
folium.Choropleth(
    geo_data=new_gdf,
    name='Average Score',
    data=new_gdf,
    columns=['SWK_ID', 'score_1'],
    key_on='properties.SWK_ID',
    fill_color='RdYlBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Score',
    reverse=True,
    highlight=True,
    show=False,
    
).add_to(m)

#Why there are no geometry data
print(new_gdf['score_1'].isna().sum())
new_gdf = new_gdf.dropna(subset=['geometry'])

# Adds the ability to click the sidewalk to see the score for that sidewalk
folium.GeoJson(
    new_gdf,
    name='Sidewalk Scores',
    tooltip=folium.features.GeoJsonTooltip(fields=['SWK_ID','score_1'], labels=True, sticky=True),
    style_function=lambda feature: {'weight': 1, 'color':'black', 'fillOpacity':0},
    popup=folium.features.GeoJsonPopup(fields=['SWK_ID','score_1'], labels=True, sticky=True),
    show=False
).add_to(m)


# Add a layer control to the map
# folium.LayerControl().add_to(m)

# Save the map as an HTML file
# m.save('maps/sidewalks_map.html')

c:\Users\xiao\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3382: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


4256


In [7]:
import json
import geojson
from shapely.geometry import shape, Point, Polygon
import geopandas as gpd
import folium

neighborhoods_gdf = gpd.read_file('../data/Climate_Ready_Boston_Social_Vulnerability.geojson')
new_gdf = gpd.read_file('../data/sidewalks_scores.json')
sidewalk_gdf = new_gdf.to_crs(neighborhoods_gdf.crs)


In [8]:
joined_gdf = gpd.sjoin(sidewalk_gdf, neighborhoods_gdf, how='left', op='within')
neighborhood_scores = joined_gdf.groupby('FID_right')['score_1'].mean().reset_index()
neighborhood_scores = neighborhood_scores.rename(columns={'FID_right': 'FID'})
neighborhoods_with_scores_gdf = neighborhoods_gdf.merge(neighborhood_scores, on='FID', how='left')
neighborhoods_with_scores_gdf = neighborhoods_with_scores_gdf.dropna(subset=['score_1'])
print(neighborhoods_with_scores_gdf)

c:\Users\xiao\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3382: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


     FID      GEOID10   AREA_SQFT  AREA_ACRES  POP100_RE  HU100_RE  TotDis  \
0      1  25025010405  3914567.54     89.8661       5522       994     470   
1      2  25025010404  1472713.92     33.8089       5817      1862     299   
2      3  25025010801  1376667.12     31.6039       2783      1899      84   
3      4  25025010702  3228780.12     74.1226       2400      1643      45   
4      5  25025010204  2741497.18     62.9361       3173      1283     131   
..   ...          ...         ...         ...        ...       ...     ...   
175  176  25025081800  4024571.84     92.3915       2898      1388     329   
176  177  25025081700  6819156.25    156.5463       3820      1599     715   
177  178  25025081500  3619583.13     83.0942       2134       905     340   
178  179  25025081400  6678367.07    153.3142       3003      1462     268   
179  180  25025081300  6246271.61    143.3947       4760      2111    1146   

     TotChild  OlderAdult  Low_to_No   LEP  POC2  MedIllnes    

In [9]:

# Add sidewalks layer to the map with color ramp based on average score
folium.Choropleth(
    geo_data=neighborhoods_with_scores_gdf,
    name='Neighborhood Colors',
    data=neighborhoods_with_scores_gdf,
    columns=['FID', 'score_1'],
    key_on='feature.properties.FID',
    fill_color='RdYlBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Score',
    bins=list(range(0, 101, 10)),
    reverse=True,
    highlight=True,
    show=False
).add_to(m)

print(neighborhoods_with_scores_gdf['score_1'].isna().sum())


# Adds the ability to click the sidewalk to see the score for that sidewalk
folium.GeoJson(
    neighborhoods_with_scores_gdf,
    name='Neighborhood Scores',
    tooltip=folium.features.GeoJsonTooltip(fields=['FID'], labels=True, sticky=True),
    style_function=lambda feature: {'weight': 1, 'color':'black', 'fillOpacity':0},
    popup=folium.features.GeoJsonPopup(fields=['score_1', 'Name'], labels=True, sticky=True),
    show=False
).add_to(m)




# Add a layer control to the map
folium.LayerControl().add_to(m)

# Save the map as an HTML file
m.save('maps/neighborhood_scores_ramps.html')

0
